In [27]:
# Dependencies
import requests
import numpy as np
import pandas as pd
from census import Census
import matplotlib.pyplot as plt

In [34]:
api_key = "a63d5bb264baf41c8c8d35a42353fe800f86bb99"
year = 2018
# Create an instance of the Census library
c = Census(
    api_key,
    year = year
)

# "B01003_001E": "Population",
# "B01002_001E": "Median Age",
# "B19013_001E": "Household Income",
# "B19301_001E": "Per Capita Income",
# "B17001_002E": "Poverty Count",
# "B15003_022E" : Education_bachelors,
# "B25077_001E" : Median_home_value

In [38]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
census_data = c.acs5.get(
    (
        "NAME",
        "B19013_001E",
        "B01003_001E",
        "B01002_001E",
        "B19301_001E",
        "B17001_002E",
        "B15003_022E",
        "B25077_001E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column renaming
census_pd = census_pd.rename(
    columns = {
        "B01003_001E": "Population",
        "B01002_001E": "Median Age",
        "B19013_001E": "Household Income",
        "B19301_001E": "Per Capita Income",
        "B17001_002E": "Poverty Count",
        "B15003_022E" : "Education_bachelors",
        "B25077_001E" : "Median_home_value",
        "NAME": "Name",
        "zip code tabulation area": "Zipcode"
    }
)

# Add a Poverty Rate column (Poverty Count / Population)
# census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"] / census_pd["Population"]


In [39]:
# Configure the final DataFrame
census_pd = census_pd[
    [
        "Zipcode",
        "Population",
        "Median Age",
        "Household Income",
        "Per Capita Income",
        "Poverty Count",
        "Poverty Rate",
        "Education_bachelors",
        "Median_home_value"
    ]
]

census_pd["Year"]=year
# Display DataFrame length and sample data
print(f"Number of rows in the DataFrame: {len(census_pd)}")
census_pd.head()

Number of rows in the DataFrame: 33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Education_bachelors,Median_home_value,Year
0,00601,17242.0,40.5,13092.0,6999.0,10772.0,62.475351,1781.0,86200.0,2018
1,00602,38442.0,42.3,16358.0,9277.0,19611.0,51.014515,3692.0,86300.0,2018
2,00603,48814.0,41.1,16603.0,11307.0,24337.0,49.856599,5888.0,122400.0,2018
3,00606,6437.0,43.3,12832.0,5943.0,4163.0,64.672984,311.0,91600.0,2018
4,00610,27073.0,42.1,19309.0,10220.0,11724.0,43.305138,2498.0,88600.0,2018


In [40]:
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
census_pd.to_csv(f"census_data_{year}.csv", encoding="utf-8", index=False)